In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .master("spark://spark-master:7077")
    .appName("notebook-test")
    .config("spark.executor.cores", "1")
    .config("spark.executor.memory", "512m")
    .config("spark.driver.memory", "512m")
    .config("spark.cores.max", "1")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/19 08:27:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sparkContext.master

'spark://spark-master:7077'

In [5]:
spark.range(1_000_000).count()

1000000

In [4]:
SparkSession.getActiveSession()

In [4]:
 spark.sql("SHOW NAMESPACES IN hive_cat").show(truncate=False)

+---------+
|namespace|
+---------+
|cleaned  |
|default  |
|features |
|graph    |
|marts    |
|raw      |
+---------+



In [6]:
spark.stop()

# Создание слоев

In [5]:
spark.sql("SHOW CATALOGS").show(truncate=False)

+-------------+
|catalog      |
+-------------+
|hive_cat     |
|spark_catalog|
+-------------+



In [6]:
 spark.sql("SHOW NAMESPACES IN hive_cat").show(truncate=False)

+---------+
|namespace|
+---------+
|cleaned  |
|default  |
|features |
|graph    |
|marts    |
|raw      |
+---------+



In [4]:
 spark.sql("create NAMESPACE if not exists hive_cat.raw")

DataFrame[]

In [5]:
 spark.sql("create NAMESPACE if not exists hive_cat.cleaned")

DataFrame[]

In [6]:
 spark.sql("create NAMESPACE if not exists hive_cat.features")

DataFrame[]

In [7]:
 spark.sql("create NAMESPACE if not exists hive_cat.marts")

DataFrame[]

In [8]:
 spark.sql("create NAMESPACE if not exists hive_cat.graph")

DataFrame[]

# Загрузка данных

In [6]:
def process_csv_to_iceberg(csv_path, table_name):
    #print(f"Reading CSV from: {csv_path}")
    
    df_csv = (spark.read
              .option("header", "true")
              .option("inferSchema", "true")
              .csv(csv_path)
    )
    
    #print(f"Writing to Iceberg table: {table_name}")
    
    (df_csv.writeTo(table_name)
     .using("iceberg")
     .createOrReplace()
    )
    
    #print(f"First 5 rows from {table_name}:")
    #spark.table(table_name).show(5, truncate=False)

In [7]:
process_csv_to_iceberg("/data/test_transaction.csv", "hive_cat.raw.test_transaction")

25/12/19 08:10:17 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/12/19 08:10:32 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/12/19 08:10:47 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/12/19 08:11:02 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/12/19 08:11:17 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/12/19 08:11:32 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

Py4JJavaError: An error occurred while calling o63.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Master removed our application: KILLED
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1247)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1$adapted(DAGScheduler.scala:1247)
	at scala.Option.foreach(Option.sca

In [13]:
process_csv_to_iceberg("/data/test_identity.csv", "hive_cat.raw.test_identity")

DataFrame[TransactionID: int, id-01: double, id-02: double, id-03: double, id-04: double, id-05: double, id-06: double, id-07: double, id-08: double, id-09: double, id-10: double, id-11: double, id-12: string, id-13: double, id-14: double, id-15: string, id-16: string, id-17: double, id-18: double, id-19: double, id-20: double, id-21: double, id-22: double, id-23: string, id-24: double, id-25: double, id-26: double, id-27: string, id-28: string, id-29: string, id-30: string, id-31: string, id-32: double, id-33: string, id-34: string, id-35: string, id-36: string, id-37: string, id-38: string, DeviceType: string, DeviceInfo: string]

In [14]:
process_csv_to_iceberg("/data/train_transaction.csv", "hive_cat.raw.train_transaction")

DataFrame[TransactionID: int, isFraud: int, TransactionDT: int, TransactionAmt: double, ProductCD: string, card1: int, card2: double, card3: double, card4: string, card5: double, card6: string, addr1: double, addr2: double, dist1: double, dist2: double, P_emaildomain: string, R_emaildomain: string, C1: double, C2: double, C3: double, C4: double, C5: double, C6: double, C7: double, C8: double, C9: double, C10: double, C11: double, C12: double, C13: double, C14: double, D1: double, D2: double, D3: double, D4: double, D5: double, D6: double, D7: double, D8: double, D9: double, D10: double, D11: double, D12: double, D13: double, D14: double, D15: double, M1: string, M2: string, M3: string, M4: string, M5: string, M6: string, M7: string, M8: string, M9: string, V1: double, V2: double, V3: double, V4: double, V5: double, V6: double, V7: double, V8: double, V9: double, V10: double, V11: double, V12: double, V13: double, V14: double, V15: double, V16: double, V17: double, V18: double, V19: dou

In [15]:
process_csv_to_iceberg("/data/train_identity.csv", "hive_cat.raw.train_identity")

DataFrame[TransactionID: int, id_01: double, id_02: double, id_03: double, id_04: double, id_05: double, id_06: double, id_07: double, id_08: double, id_09: double, id_10: double, id_11: double, id_12: string, id_13: double, id_14: double, id_15: string, id_16: string, id_17: double, id_18: double, id_19: double, id_20: double, id_21: double, id_22: double, id_23: string, id_24: double, id_25: double, id_26: double, id_27: string, id_28: string, id_29: string, id_30: string, id_31: string, id_32: double, id_33: string, id_34: string, id_35: string, id_36: string, id_37: string, id_38: string, DeviceType: string, DeviceInfo: string]

# Запись из raw в cleaned

In [16]:
spark.conf.set("spark.sql.shuffle.partitions", "64")
spark.conf.set("spark.sql.files.maxRecordsPerFile", "500000")
#spark.conf.set("parquet.enable.dictionary", "false")

In [17]:
from pyspark.sql import SparkSession, functions as F
import datetime as dt

In [18]:
def load_raw_to_cleaned(spark: SparkSession) -> None:

    train_txn = spark.table("hive_cat.raw.train_transaction")
    train_id = spark.table("hive_cat.raw.train_identity")
    test_txn = spark.table("hive_cat.raw.test_transaction")
    test_id = spark.table("hive_cat.raw.test_identity")

    base_epoch = int(dt.datetime(2017, 1, 1, 0, 0, 0).timestamp())

    def build_view(df_txn, df_id, dataset_label: str, has_target: bool):
        df = (df_txn.alias("t")
            .join(df_id.alias("i"), on="TransactionID", how="left")
            .withColumn(
                "timestamp",
                F.to_timestamp(F.col("TransactionDT").cast("long") + F.lit(base_epoch))
            )
            .withColumnRenamed("TransactionID", "transaction_id")
            .withColumnRenamed("TransactionAmt", "amount")
        )

        if has_target and "isFraud" in df.columns:
            df = df.withColumnRenamed("isFraud", "is_fraud")
        else:
            df = df.withColumn("is_fraud", F.lit(None).cast("int"))

        df = (df
            .withColumn("customer_id", F.col("card1").cast("string"))
            .withColumn(
                "card_id",
                F.concat_ws(
                    "-",
                    F.col("card1").cast("string"),
                    F.col("card2").cast("string"),
                    F.col("card3").cast("string"),
                )
            )
            .withColumn("country", F.col("addr2").cast("string"))
            .withColumn("region", F.col("addr1").cast("string"))
            .withColumn("device_id", F.coalesce(F.col("DeviceInfo"), F.lit("unknown")))
            .withColumn(
                "channel",
                F.when(F.col("DeviceType") == "mobile", "MOBILE")
                 .when(F.col("DeviceType") == "desktop", "WEB")
                 .otherwise("UNKNOWN")
            )
            .withColumn("email", F.col("P_emaildomain"))
            .withColumn("dataset", F.lit(dataset_label))
            .withColumn("event_date", F.to_date(F.col("timestamp")))
        )

        important_cols = [
            "transaction_id",
            "timestamp",
            "event_date",
            "amount",
            "customer_id",
            "card_id",
            "country",
            "region",
            "device_id",
            "channel",
            "email",
            "is_fraud",
            "dataset",
        ]

        cols_to_keep = [c for c in important_cols if c in df.columns]
        df = df.select(*cols_to_keep)

        return df

    train_clean = build_view(train_txn, train_id, dataset_label="train", has_target=True)
    test_clean  = build_view(test_txn,  test_id,  dataset_label="test",  has_target=False)

    #spark.sql("CREATE NAMESPACE IF NOT EXISTS hive_cat.cleaned")

    (
        train_clean
        .repartition("event_date")  # repartition(64, "event_date")
        .writeTo("hive_cat.cleaned.transactions")
        .using("iceberg")
        .tableProperty("write.format.default", "parquet")
        .tableProperty("format-version", "2")
        .partitionedBy("event_date")
        .createOrReplace()
    )
    
    (
        test_clean
        .repartition("event_date")
        .writeTo("hive_cat.cleaned.transactions")
        .append()
    )

In [19]:
if __name__ == "__main__":
    load_raw_to_cleaned(spark)

# Запись из cleaned в слой features

In [20]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window

In [21]:
def build_features(spark: SparkSession) -> None:
    df = spark.table("hive_cat.cleaned.transactions")

    df = df.filter(F.col("customer_id").isNotNull())

    ts_long = F.col("timestamp").cast("long")

    w_cust_1d = (
        Window.partitionBy("customer_id")
        .orderBy(ts_long)
        .rangeBetween(-86400, 0)
    )

    w_cust_7d = (
        Window.partitionBy("customer_id")
        .orderBy(ts_long)
        .rangeBetween(-86400 * 7, 0)
    )

    w_cust_order = (
        Window.partitionBy("customer_id")
        .orderBy("timestamp")
    )

    df_feat = (
        df
        .withColumn("hour", F.hour("timestamp"))
        .withColumn(
            "is_night",
            F.when((F.col("hour") < 6) | (F.col("hour") >= 23), F.lit(1)).otherwise(F.lit(0))
        )
        .withColumn("log_amount", F.log1p(F.col("amount")))
        .withColumn("cust_txn_cnt_1d", F.count("*").over(w_cust_1d))
        .withColumn("cust_amt_sum_1d", F.sum("amount").over(w_cust_1d))
        .withColumn("cust_txn_cnt_7d", F.count("*").over(w_cust_7d))
        .withColumn("cust_amt_sum_7d", F.sum("amount").over(w_cust_7d))
        .withColumn("prev_timestamp", F.lag("timestamp").over(w_cust_order))
        .withColumn(
            "secs_since_prev_txn",
            (F.col("timestamp").cast("long") - F.col("prev_timestamp").cast("long"))
        )
    )

    cols = [
        "transaction_id",
        "timestamp",
        "event_date",
        "dataset",
        "customer_id",
        "card_id",
        "country",
        "region",
        "device_id",
        "channel",
        "email",
        "is_fraud",
        "amount",
        "log_amount",
        "hour",
        "is_night",
        "cust_txn_cnt_1d",
        "cust_amt_sum_1d",
        "cust_txn_cnt_7d",
        "cust_amt_sum_7d",
        "secs_since_prev_txn",
    ]

    df_feat = df_feat.select(*cols)

    (
        df_feat
        .repartition("event_date")
        .writeTo("hive_cat.features.transaction_features")
        .using("iceberg")
        .tableProperty("write.format.default", "parquet")
        .tableProperty("format-version", "2")
        .partitionedBy("event_date")
        .createOrReplace()
    )

In [23]:
if __name__ == "__main__":
    build_features(spark)

# Данные для модели

## LightGBM

### export train

In [8]:
from pyspark.sql import SparkSession, functions as F

In [13]:
S3_BASE = "s3a://warehouse/features.db/lgbm_data"

def export_train_dataset(spark: SparkSession) -> None:
    df = spark.table("hive_cat.features.trans_gnn")

    df_train = df.filter(
        (F.col("dataset") == "train") & F.col("is_fraud").isNotNull()
    )

    output_path = f"{S3_BASE}/train"

    (
        df_train
        # .repartition(1)
        .write
        .mode("overwrite")
        .parquet(output_path)
    )

In [14]:
if __name__ == "__main__":
    export_train_dataset(spark)

### export test

In [15]:
S3_BASE = "s3a://warehouse/features.db/lgbm_data"

def export_test_dataset(spark: SparkSession) -> None:
    df = spark.table("hive_cat.features.trans_gnn")

    df_test = df.filter(F.col("dataset") == "test")

    output_path = f"{S3_BASE}/test"

    (
        df_test
        # .repartition(1)
        .write
        .mode("overwrite")
        .parquet(output_path)
    )

In [16]:
if __name__ == "__main__":
    export_test_dataset(spark)

## GNN

### Подготовка ребер и узлов

In [2]:
from pyspark.sql import SparkSession, functions as F

In [25]:
def export_gnn_graph(spark: SparkSession,src_table: str = "hive_cat.features.transaction_features",ns: str = "hive_cat.graph") -> None:
    spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {ns}")

    df = spark.table(src_table)

    df_all = (
        df
        .withColumn("transaction_id_str", F.col("transaction_id").cast("string"))
        .withColumn("customer_id_str",   F.col("customer_id").cast("string"))
        .withColumn("card_id_str",       F.col("card_id").cast("string"))
        .withColumn("device_id_str",     F.col("device_id").cast("string"))
        .withColumn("email_str",         F.col("email").cast("string"))
    )

    df_train = (
        df_all
        .filter(F.col("dataset") == "train")
        .withColumn("label", F.col("is_fraud").cast("int"))
    )

    def write_iceberg(df_out, table_name: str, mode: str = "createOrReplace"):
        full = f"{ns}.{table_name}"
        w = df_out.writeTo(full).using("iceberg") \
            .tableProperty("write.format.default", "parquet") \
            .tableProperty("format-version", "2")
        if mode == "append":
            w.append()
        else:
            w.createOrReplace()

    # ---------- nodes ----------
    customers = (
        df_all.select(F.col("customer_id_str").alias("customer_id"))
        .where(F.col("customer_id_str").isNotNull())
        .distinct()
    )
    write_iceberg(customers, "nodes_customers")

    cards = (
        df_all.select(F.col("card_id_str").alias("card_id"))
        .where(F.col("card_id_str").isNotNull())
        .distinct()
    )
    write_iceberg(cards, "nodes_cards")

    devices = (
        df_all.select(F.col("device_id_str").alias("device_id"))
        .where(F.col("device_id_str").isNotNull())
        .distinct()
    )
    write_iceberg(devices, "nodes_devices")

    emails = (
        df_all.select(F.col("email_str").alias("email"))
        .where(F.col("email_str").isNotNull())
        .distinct()
    )
    write_iceberg(emails, "nodes_emails")

    txns = (
        df_all.select(F.col("transaction_id_str").alias("transaction_id"))
        .where(F.col("transaction_id_str").isNotNull())
        .distinct()
    )
    write_iceberg(txns, "nodes_transactions")

    # ---------- labales (only train) ----------
    txn_labels = (
        df_train
        .select(
            F.col("transaction_id_str").alias("transaction_id"),
            F.col("label"),
        )
        .where(F.col("transaction_id_str").isNotNull())
    )
    write_iceberg(txn_labels, "labels_transactions")

    # ---------- edges ----------
    edges_cust_card = (
        df_all.select(
            F.col("customer_id_str").alias("customer_id"),
            F.col("card_id_str").alias("card_id"),
        )
        .where(F.col("customer_id_str").isNotNull() & F.col("card_id_str").isNotNull())
        .distinct()
    )
    write_iceberg(edges_cust_card, "edges_customer_card")
    edges_card_device = (
        df_all.select(
            F.col("card_id_str").alias("card_id"),
            F.col("device_id_str").alias("device_id"),
        )
        .where(F.col("card_id_str").isNotNull() & F.col("device_id_str").isNotNull())
        .distinct()
    )
    write_iceberg(edges_card_device, "edges_card_device")

    edges_card_email = (
        df_all.select(
            F.col("card_id_str").alias("card_id"),
            F.col("email_str").alias("email"),
        )
        .where(F.col("card_id_str").isNotNull() & F.col("email_str").isNotNull())
        .distinct()
    )
    write_iceberg(edges_card_email, "edges_card_email")

    edges_cust_txn = (
        df_all.select(
            F.col("customer_id_str").alias("customer_id"),
            F.col("transaction_id_str").alias("transaction_id"),
        )
        .where(F.col("customer_id_str").isNotNull() & F.col("transaction_id_str").isNotNull())
        .distinct()
    )
    write_iceberg(edges_cust_txn, "edges_customer_txn")

    edges_txn_device = (
        df_all.select(
            F.col("transaction_id_str").alias("transaction_id"),
            F.col("device_id_str").alias("device_id"),
        )
        .where(F.col("transaction_id_str").isNotNull() & F.col("device_id_str").isNotNull())
        .distinct()
    )
    write_iceberg(edges_txn_device, "edges_txn_device")

In [26]:
if __name__ == "__main__":
    export_gnn_graph(spark)

GNN graph export finished -> namespace hive_cat.graph


### import/join gnn features

In [2]:
from pyspark.sql import SparkSession, functions as F

In [5]:
def import_gnn_to_features(
    spark: SparkSession,
    gnn_path: str = "s3a://warehouse/features.db/gnn_features/txn_gnn_features.parquet",) -> None:
    df_gnn = spark.read.parquet(gnn_path)

    df_gnn = df_gnn.withColumn("transaction_id_str", F.col("transaction_id").cast("string")) \
                   .drop("transaction_id")

    df_feat = spark.table("hive_cat.features.transaction_features") \
                   .withColumn("transaction_id_str", F.col("transaction_id").cast("string"))

    matched = df_feat.join(df_gnn, on="transaction_id_str", how="inner").count()
    #print("Matched rows by transaction_id_str:", matched)

    df_join = (
        df_feat.join(df_gnn, on="transaction_id_str", how="left")
               .drop("transaction_id_str")
    )

    fill_map = {"gnn_score": 0.0}
    for i in range(1, 17):
        col = f"gnn_emb_{i}"
        if col in df_join.columns:
            fill_map[col] = 0.0

    df_join = df_join.na.fill(fill_map)

    (
        df_join
        .repartition("event_date")
        .writeTo("hive_cat.features.trans_gnn")
        .using("iceberg")
        .tableProperty("write.format.default", "parquet")
        .tableProperty("format-version", "2")
        .partitionedBy("event_date")
        .createOrReplace()
    )

In [6]:
if __name__ == "__main__":
    import_gnn_to_features(spark)

Matched rows by transaction_id_str: 1097231


25/12/18 13:11:58 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

# Запись в marts

In [17]:
from pyspark.sql import SparkSession, functions as F

In [18]:
def import_scored_to_marts(
    spark: SparkSession,
    scored_path: str = "s3a://warehouse/features.db/lgbm_data/scored/test_scored.parquet",) -> None:
    df_scored = spark.read.parquet(scored_path)
    
    df_scored = (
        df_scored
        .withColumn("event_date", F.to_date(F.col("event_date")))
        .withColumn("fraud_score", F.col("fraud_score").cast("double"))
        .withColumn("fraud_label", F.col("fraud_label").cast("int"))
    )

    spark.sql("CREATE NAMESPACE IF NOT EXISTS hive_cat.marts")

    (
        df_scored
        .repartition("event_date")
        .writeTo("hive_cat.marts.scored_transactions")
        .using("iceberg")
        .tableProperty("write.format.default", "parquet")
        .tableProperty("format-version", "2")
        .partitionedBy("event_date")
        .createOrReplace()
    )

In [19]:
if __name__ == "__main__":
    import_scored_to_marts(spark)

# Кластеризация

In [21]:
from pyspark.sql import SparkSession, functions as F
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans, BisectingKMeans

In [22]:
df = spark.table("hive_cat.marts.scored_transactions") \
    .select(
        "transaction_id",
        "log_amount",
        F.log10(F.col("secs_since_prev_txn") + 1).alias("log_secs_prev"),
        F.log10(F.col("cust_txn_cnt_7d") + 1).alias("log_txn_cnt_7d"),
        "hour",
        "is_night",
        "fraud_score",
        "fraud_label"
    ) \
    .na.fill(0)

In [23]:
assembler = VectorAssembler(
    inputCols=["log_amount", "log_secs_prev", "log_txn_cnt_7d", "hour", "is_night"],
    outputCol="features_raw"
)

df2 = assembler.transform(df)

In [24]:
scaler = StandardScaler(inputCol="features_raw", outputCol="features")
model_scaler = scaler.fit(df2)
df3 = model_scaler.transform(df2)

In [25]:
k = 6
kmeans = BisectingKMeans(k=k, featuresCol="features", predictionCol="cluster")
model_kmeans = kmeans.fit(df3)

df_clusters = model_kmeans.transform(df3)

25/12/18 14:55:58 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
                                                                                

In [26]:
(
    df_clusters
    .select(
        "transaction_id",
        "cluster",
        "log_amount",
        "log_secs_prev",
        "log_txn_cnt_7d",
        "hour",
        "is_night",
        "fraud_score",
        "fraud_label"
    )
    .writeTo("hive_cat.marts.scored_transactions_clustered")
    .using("iceberg")
    .createOrReplace()
)